In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet, ElasticNetCV, Ridge, RidgeCV
from sklearn.metrics import mean_squared_error
np.warnings.filterwarnings("ignore")
import statsmodels.api as sm
np.warnings.resetwarnings()
import os

In [2]:
def getElasticNetBestHyperparams(X, y):
    l1_ratios = [.1, .5, .7, .9, .95, .99, 1]
    min_mse = 1
    best_l1_ratio = 1
    best_alpha = 1
    params_list = []
    for ratio in l1_ratios:
#         encv = ElasticNetCV(l1_ratio = ratio, n_alphas = 100, cv = 5, verbose = 1, precompute = True, max_iter=2500, n_jobs = -1)
        encv = ElasticNetCV(l1_ratio = ratio, n_alphas = 10, cv = 3, verbose = 1, precompute = True, max_iter=2500, n_jobs = -1)
        encv.fit(X, y)
        n_nonzeros = (encv.coef_ != 0).sum()
        _mse = np.mean(encv.mse_path_, axis=1)[np.where(encv.alphas_ == encv.alpha_)[0][0]]
        if (ratio == l1_ratios[0] or _mse < min_mse):
            min_mse = _mse
            best_l1_ratio = ratio
            best_alpha = encv.alpha_
        print("ratio(%e) -- n: %d -- alpha: %f -- mse: %f" % (ratio, n_nonzeros, encv.alpha_, _mse))
        if n_nonzeros == 0:
            break
        params_list.append(tuple([n_nonzeros, _mse, ratio, encv.alpha_]))
    # select the simplest model whose mean-squared error is 'not so bad'
    sorted_params_list = sorted(params_list)
    for param_tuple in sorted_params_list:
        if param_tuple[1] - min_mse <= 0.1:
            print("for target_gene " + y.name + ", nonzero_coeffs_num, MSE, l1-ratio, alpha:")
            print(param_tuple)
            return param_tuple
    return tuple([0, 0, 0, 0])

In [3]:
def getElasticNetSelectedFeatures(X, y, alpha, l1_ratio):
    enet = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, max_iter=2000)
    enet.fit(X, y)
    selected_features = []
    for ix in range(enet.coef_.shape[0]):
        if (enet.coef_[ix] != 0):
            selected_features.append(ix)
    return list(X.columns[selected_features])

In [4]:
def getOlsSelectedFeatures(X, y, threshold):
    X = sm.add_constant(X)
    ols_model_results = sm.OLS(y, X).fit()
    p_values = ols_model_results.pvalues
    p_values = p_values.where(p_values < threshold)
    p_values.dropna(inplace=True)
    return list(p_values.index)[1:]

In [5]:
def getRidgeRegressionBestHyperparams(X, y):
    ridge_cv = RidgeCV(alphas=np.linspace(.0001,1,100), cv=10)  # check this
    ridge_cv.fit(X, y)
    best_alpha = ridge_cv.alpha_
    y_pred = ridge_cv.predict(X)
    rmse_train = np.sqrt(mean_squared_error(y, y_pred))
    r2_train = ridge_cv.score(X, y)
    return tuple([best_alpha, rmse_train, r2_train])  # (best alpha, rmse_train, r2_train)

In [6]:
def getAdjustedR2(R2, num_samples, num_features):
    adjusted_R2 = (1-R2) * (num_samples-1) / (num_samples-num_features-1)
    return adjusted_R2

In [7]:
def trainRidgeRegressionModel(X, y):
    # hold-out a small percentage of the samples as test set
    num_total_samples = X.shape[0]
    num_features = X.shape[1]
    test_set_size = int(0.2*num_total_samples)
    num_training_samples = num_total_samples - test_set_size
    test_row_ix = X.index[list(np.random.choice(a=num_total_samples, replace=False, size=test_set_size))]
    X_test = X.loc[test_row_ix]
    y_test = y[test_row_ix]
    X_train = X.drop(test_row_ix, axis=0, inplace=False)
    y_train = y.drop(test_row_ix, inplace=False)
    alpha, rmse_train, r2_train = getRidgeRegressionBestHyperparams(X_train, y_train)
    adj_r2_train = getAdjustedR2(r2_train, X_train.shape[0], X_train.shape[1])
    ridge_reg_model = Ridge(fit_intercept=True, alpha=alpha)
    ridge_reg_model.fit(X_train, y_train)
    r2_test = ridge_reg_model.score(X_test, y_test)
    adj_r2_test = getAdjustedR2(r2_test, X_test.shape[0], X_test.shape[1])
    y_test_pred = ridge_reg_model.predict(X_test)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    return tuple([alpha, rmse_train, r2_train, adj_r2_train, rmse_test, r2_test, adj_r2_test, test_set_size])

In [8]:
def runLinearRegressionPipeline(X, y, output_file_name):
    temp_file = 'temp.txt'
    eve_genes = ['hb', 'Kr', 'gt']
    # create new file for this run
    with open(temp_file, "w") as fp:
        # feature selection using Elastic Net
        _, _, l1_ratio, alpha = getElasticNetBestHyperparams(X, y)
        fp.write("l1_ratio:" + str(l1_ratio) + "\n");
        fp.write("alpha:" + str(alpha) + "\n")
        if l1_ratio == 0:
            print("no features selected for " + output_file_name)
            fp.write("0 features selected by ElasticNet")
        else:
            select_features_1 = getElasticNetSelectedFeatures(X, y, alpha, l1_ratio)
            # if target gene is 'eve'
            if y.name == 'eve':
                for gene in eve_genes:
                    if gene not in select_features_1:
                        select_features_1.append(gene)
            fp.write("select_features_1:")
            for feature in select_features_1:
                fp.write(feature + ",")
            fp.write("\n")
            X_reduced_1 = X[select_features_1]
            # feature selection using OLS
            select_features_2 = getOlsSelectedFeatures(X_reduced_1, y, 0.05)
            if len(select_features_2) == 0:
                X_reduced_2 = X_reduced_1
            else:
                if y.name == 'eve':  # if target gene is 'eve'
                    for gene in eve_genes:
                        if gene not in select_features_2:
                            select_features_2.append(gene)
                X_reduced_2 = X_reduced_1[select_features_2]
            fp.write("select_features_2:")
            for feature in select_features_2:
                fp.write(feature + ",")
            fp.write("\n")
            # train Ridge regression model with final set of features
            alpha, rmse_train, r2_train, adj_r2_train, rmse_test, r2_test, adj_r2_test, test_set_size = trainRidgeRegressionModel(X_reduced_2, y)
            fp.write("ridge_alpha:" + str(alpha) + "\n")
            fp.write("rmse_train:" + str(rmse_train) + "\n")
            fp.write("r2_train:" + str(r2_train) + "\n")
            fp.write("adj_r2_train:" + str(adj_r2_train) + "\n")
            fp.write("test_set_size:" + str(test_set_size) + "\n")
            fp.write("rmse_test:" + str(rmse_test) + "\n")
            fp.write("r2_test:" + str(r2_test) + "\n")
            fp.write("adj_r2_test:" + str(adj_r2_test) + "\n")
    os.rename(temp_file, output_file_name)

In [9]:
def modellingUsingCompleteEmbryo(cells_genes_df, genes_set, path):
    for target_gene in genes_set:
        print("modelling using complete embryo for target gene " + target_gene)
        y = cells_genes_df[target_gene].copy(deep=True)
        X = cells_genes_df.drop({target_gene}, axis=1)
        output_file_name = path + target_gene + "_complete.txt"
        # check if file already exists, if it does, then continue to next target_gene
        if (os.path.isfile(output_file_name)):
            print(output_file_name + " already exists!")
            continue
        runLinearRegressionPipeline(X, y, output_file_name)

In [10]:
def modellingPerCluster(cells_genes_df, cell_cluster_labels, gene_cluster_map, path):
    cells_genes_df["cluster_labels"] = cell_cluster_labels
    for target_gene, clusters in gene_cluster_map.items():
        X_cluster = cells_genes_df[cells_genes_df["cluster_labels"].isin(clusters)]
        print("modelling using cluster(s) " + ','.join(clusters) + " for target gene " + target_gene)
        y = X_cluster[target_gene].copy(deep=True)
        X = X_cluster.drop({target_gene, "cluster_labels"}, axis=1)
        print('original df shape: ', cells_genes_df.shape)
        print('cluster df shape: ', X_cluster.shape)
        output_file_name = path + target_gene + "_cluster_" + '_'.join(clusters) + ".txt"
        if (os.path.isfile(output_file_name)):
            print(output_file_name + " already exists!")
            continue
        runLinearRegressionPipeline(X, y, output_file_name)

In [11]:
def getTopGenesPerCluster():
    cluster_top_genes = {}
    cluster_top_genes_file = "cluster_top_genes.csv"
    with open(cluster_top_genes_file, "r") as fp:
        for line in fp:
            cluster_no = int(line.split(", ")[0].strip())
            top_genes = line.split(", ")[1:]
            top_genes = [gene.strip() for gene in top_genes]
            cluster_top_genes[cluster_no] = top_genes
    return cluster_top_genes

In [12]:
def getClusterLabels(X):
    cell_cluster_labels = []
    for cell_name in list(X.index):
        cell_cluster_labels.append(cell_name.split("_")[1])
    return cell_cluster_labels

In [13]:
def getTargetGenesWithClusters():
    target_gene_cluster_map = {
        'twe': ['4', '2'],
        'sna': ['4', '2'],
        'htl': ['4'],
        'tin': ['4'],
        'eve': ['3'],
        'brk': ['3', '5'],
        'vnd': ['3'],
        'rho': ['6', '11'],
        'sli': ['3', '6'],
        'pnt': ['3', '5'],
        'ind': ['3'],
        'sog': ['3', '8'],
        'ths': ['3'],
        'zen': ['6', '10'],
        'pnr': ['6', '3'],
        'shn': ['4', '3'],
        'tup': ['6', '10'],
        'ush': ['6', '10']
    }
    return target_gene_cluster_map

In [14]:
def main():
    data_file_name = "dge_normalized.txt"
    gene_sc_df = pd.read_csv(data_file_name, delimiter='\t', header=0)
    cells_genes_df = gene_sc_df.T
    cell_cluster_labels = getClusterLabels(cells_genes_df)
    cluster_top_genes = getTopGenesPerCluster()
    genes_set = set()
    gene_cluster_map = {}
    for cluster, top_genes in cluster_top_genes.items():
        genes_set.update(top_genes)
        for gene in top_genes:
            if gene not in gene_cluster_map:
                gene_cluster_map[gene] = [str(cluster)]
            else:
                gene_cluster_map[gene].append(str(cluster))
    modellingUsingCompleteEmbryo(cells_genes_df, genes_set)
    modellingPerCluster(cells_genes_df, cell_cluster_labels, gene_cluster_map)

In [16]:
def new_main():
    data_file_name = "dge_normalized.txt"
    gene_sc_df = pd.read_csv(data_file_name, delimiter='\t', header=0)
    cells_genes_df = gene_sc_df.T
    cell_cluster_labels = getClusterLabels(cells_genes_df)
    target_gene_cluster_map = getTargetGenesWithClusters()
    genes_set = set(target_gene_cluster_map.keys())
    # run the pipeline for target gene using all the cells of the embryo
    modellingUsingCompleteEmbryo(cells_genes_df, genes_set, "using_embryo_2/")
    # run the pipeline for target gene using cells from the designated clusters
    modellingPerCluster(cells_genes_df, cell_cluster_labels, target_gene_cluster_map, "using_clusters_2/")

In [17]:
new_main()

modelling using complete embryo for target gene ths


..............................[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  1.7min finished


ratio(1.000000e-01) -- n: 231 -- alpha: 1.145137 -- mse: 2.056864


..............................[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  1.1min finished


ratio(5.000000e-01) -- n: 38 -- alpha: 0.493425 -- mse: 2.086462


...

..............................[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   10.7s finished


ratio(9.900000e-01) -- n: 65 -- alpha: 0.093211 -- mse: 2.213634


..............................[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   11.1s finished


ratio(1.000000e+00) -- n: 82 -- alpha: 0.004283 -- mse: 2.221990
for target_gene ush, nonzero_coeffs_num, MSE, l1-ratio, alpha:
(213, 1.6904481070230908, 0.1, 0.042831955025569239)


/Users/pooja/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/pooja/anaconda/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:1353: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


In [ ]:
# please ignore this cell; it's only for testing purposes
def main2():
    data_file_name = "dge_normalized.txt"
    gene_sc_df = pd.read_csv(data_file_name, delimiter='\t', header=0)
    cells_genes_df = gene_sc_df.T
    cluster_top_genes = {1:["Act87E"]}
    print("calling modellingUsingCompleteEmbryo")
    modellingUsingCompleteEmbryo(cells_genes_df, cluster_top_genes)